In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/master_thesis/Resnet
!pip install carbontracker

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from tqdm import tqdm

from model import resnet34
from carbontracker.tracker import CarbonTracker

In [ ]:
def evaluate_model_on_testset(model, device, test_path):
    # Assume the test data is structured in a similar way to the training data
    test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    test_dataset = datasets.ImageFolder(root=test_path, transform=test_transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

    model.eval()
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')




def exponential_moving_average(data, alpha):
    """Calculate the exponential moving average using the specified alpha value."""
    return pd.Series(data).ewm(alpha=alpha, adjust=False).mean().values

def plot_and_save_training_results(train_loss_values, val_loss_values, train_accuracy_values, val_accuracy_values, save_path, filename):
    if len(train_loss_values) == 0 or len(val_loss_values) == 0:
        print("No data to plot.")
        return

    # Calculate smoothed data
    smoothed_train_loss = exponential_moving_average(train_loss_values, 0.5)
    smoothed_val_loss = exponential_moving_average(val_loss_values, 0.5)
    smoothed_train_accuracy = exponential_moving_average(train_accuracy_values, 0.5)
    smoothed_val_accuracy = exponential_moving_average(val_accuracy_values, 0.5)

    epochs_range = range(1, len(train_loss_values) + 1)
    plt.figure(figsize=(12, 6))

    # Plot training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, smoothed_train_loss, label='Training Loss', color='blue')
    plt.plot(epochs_range, smoothed_val_loss, label='Validation Loss', color='red')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, smoothed_train_accuracy, label='Training Accuracy', color='blue')
    plt.plot(epochs_range, smoothed_val_accuracy, label='Validation Accuracy', color='red')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.ylim([0, 1])

    plt.tight_layout()
    plt.savefig(os.path.join(save_path, filename))
    plt.show()


def count_parameters(model, trainable_only=True):
    """Calculate the total number of parameters in the model. If trainable_only is True, only count the trainable parameters."""
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())


In [ ]:
class NewFC(nn.Module):
    def __init__(self, in_features):
        super(NewFC, self).__init__()
        self.flatten = nn.Flatten()  # Flatten layer
        self.fc1 = nn.Linear(in_features, 512)  # Fully connected layer from in_features to 512 features
        self.relu = nn.ReLU()  # Activation function
        self.dropout = nn.Dropout(0.5)  # Dropout with a rate of 50%
        self.fc2 = nn.Linear(512, 2)  # Final layer mapping to 2 classes

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using {} device.".format(device))

    data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
        "val": transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

    # Change the path to the desired dataset
    image_path = "/content/drive/MyDrive/master_thesis/Datasets/BreastMNIST"
    # image_path = "/content/drive/MyDrive/master_thesis/Datasets/Herlev"
    # image_path = "/content/drive/MyDrive/master_thesis/Datasets/ISIC"
    assert os.path.exists(image_path), "{} path does not exist.".format(image_path)

    train_dataset = datasets.ImageFolder(root=os.path.join(image_path, "train"),
                                         transform=data_transform["train"])
    train_num = len(train_dataset)

    # Save class-to-index mapping
    class_dict = train_dataset.class_to_idx
    class_dict_rev = dict((val, key) for key, val in class_dict.items())
    json_str = json.dumps(class_dict_rev, indent=4)
    with open('class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    batch_size = 16
    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
    print('Using {} dataloader workers every process'.format(nw))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=nw)

    validate_dataset = datasets.ImageFolder(root=os.path.join(image_path, "val"),
                                            transform=data_transform["val"])
    val_num = len(validate_dataset)
    validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                                  batch_size=batch_size, shuffle=False,
                                                  num_workers=nw)

    print("using {} images for training, {} images for validation.".format(train_num, val_num))

    net = resnet34()

    # Transfer Learning: Load pre-trained weights
    model_weight_path = "./resnet34-pre.pth"
    assert os.path.exists(model_weight_path), "file {} does not exist.".format(model_weight_path)
    net.load_state_dict(torch.load(model_weight_path, map_location='cpu', weights_only=True))

    ######
    # Structure A: Basic ResNet34 with modified output layer (choose one structure at a time)
    in_channel = net.fc.in_features
    net.fc = nn.Linear(in_channel, 2)  # Output layer changed to classify 2 classes
    net.to(device)

    ######
    # Structure B: Add a new fully connected layer without replacing the existing one
    # Uncomment the following block if you want to test Structure B:
    # in_features = net.fc.in_features
    # net.fc = nn.Sequential(
    #     nn.Linear(in_features, 1000),  # Original fully connected layer
    #     nn.ReLU(),
    #     nn.Linear(1000, 2)  # New layer from 1000 classes to 2 classes
    # )
    # net.to(device)

    ######
    # Structure C: Replace the original fully connected layer with a new one
    # Uncomment the following block if you want to test Structure C:
    # in_features = net.fc.in_features
    # net.fc = NewFC(in_features)  # Replace with custom fully connected layer
    # net.to(device)

    # Calculate and print model parameters
    total_params = count_parameters(net, trainable_only=False)
    trainable_params = count_parameters(net, trainable_only=True)
    print(f"Total parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")

    loss_function = nn.CrossEntropyLoss()
    params = [p for p in net.parameters() if p.requires_grad]
    optimizer = optim.Adam(params, lr=0.0001)

    epochs = 200
    best_acc = 0.0
    save_path = './resnet34_fine_tuning_breastmnist_structureA.pth'  # Modify based on structure
    train_steps = len(train_loader)

    tracker = CarbonTracker(epochs=epochs)

    train_loss_values = []
    train_accuracy_values = []
    val_loss_values = []
    val_accuracy_values = []

    for epoch in range(epochs):
        tracker.epoch_start()

        net.train()
        running_loss = 0.0
        correct = 0
        total = 0
        train_bar = tqdm(train_loader, file=sys.stdout)

        for step, data in enumerate(train_bar):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(images)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, epochs, loss.item())

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct / total
        train_loss_values.append(train_loss)
        train_accuracy_values.append(train_accuracy)

        net.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            val_bar = tqdm(validate_loader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs = net(val_images)
                val_loss += loss_function(outputs, val_labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += val_labels.size(0)
                correct += (predicted == val_labels).sum().item()

        val_loss /= len(validate_loader.dataset)
        val_accuracy = correct / total
        val_loss_values.append(val_loss)
        val_accuracy_values.append(val_accuracy)

        print('[epoch %d] train_loss: %.3f, train_accuracy: %.3f, val_loss: %.3f, val_accuracy: %.3f' %
              (epoch + 1, train_loss, train_accuracy, val_loss, val_accuracy))

        if val_accuracy > best_acc:
            best_acc = val_accuracy
            torch.save(net.state_dict(), save_path)

        tracker.epoch_end()

    tracker.stop()
    print('Finished Training')

    # Plot and save training results
    plot_and_save_training_results(
        train_loss_values,
        val_loss_values,
        train_accuracy_values,
        val_accuracy_values,
        "/content/drive/MyDrive/master_thesis/Results",
        "fine_tuning_breastmnist_structureA.png"
    )

    # Test the model
    test_path = "/content/drive/MyDrive/master_thesis/Datasets/BreastMNIST/test"
    # test_path = "/content/drive/MyDrive/master_thesis/Datasets/Herlev/test"
    # test_path = "/content/drive/MyDrive/master_thesis/Datasets/ISIC/test"
    evaluate_model_on_testset(net, device, test_path)


if __name__ == '__main__':
    main()
